In [299]:
import pandas as pd
import numpy as np

df_1 = pd.read_csv('/Users/iwatashun/Desktop/datadiet/datadiet_database - health_record_d1.csv')
df_d2 = pd.read_csv('/Users/iwatashun/Desktop/datadiet/datadiet_database - ideal_nutrition_d2.csv')
df_3 = pd.read_csv('/Users/iwatashun/Desktop/datadiet/datadiet_database - menu_nutrition_d3.csv')

df_2 = df_d2.copy()
df_2 = df_2.head(10)
df_2 = df_2.drop(df_2.iloc[:, 16:], axis=1)
variables = df_2_male.columns
less = ['saturated_fat (g)','trans_fat (g)','cholesterol (mg)', 'sodium (mg)','total_sugars (g)']
more = ['vitamin_d (mcg)','calcium (mg)','protein (g)']
for i in variables:
    if i in ['resident_id','calories','water (mL)','dietary_fiber (g)','potassium (mg)','iron (mg)']:
        df_2[i] = pd.to_numeric(df_2[i])
    else:
        df_2[i] = df_2[i].astype(str)
        if i in less or i in more:
            num = []
            for j in range(len(df_2[i])):
                num.append(float(df_2[i][j].split()[1]))
            df_2[i] = num
        elif i == 'total_carbs (g)':
            num = []
            for j in range(len(df_2[i])):
                a_1 = float(df_2['total_carbs (g)'][j].split('-')[0])
                a_2 = float(df_2['total_carbs (g)'][j].split('-')[1])
                ave = (a_1 + a_2) / 2
                num.append(ave)
            df_2[i] = num             

male = df_1[df_1['gender'] == 'M']
df_2['resident_id'] = df_2['resident_id'].astype(str).astype(int)
m = df_2['resident_id'].isin(list(male['resident_id']))
df_2_male = df_2[m]
df_2_female = df_2[~m]


from statistics import mean

male_standard = {}
female_standard = {}
for i in variables:
    if i == 'resident_id' or i == 'restrictions':
        pass
    else:
        male_standard[i] = mean(df_2_male[i])
        female_standard[i] = mean(df_2_female[i])
        
#choosing the 3 options per meal
day = {}
meal_type = ['bf','lun','din']
for i in meal_type:
    combo = []
    for j in range(3):
        combo.append(df_3['Menu Items'].sample(n = 3))
    day[i] = combo

In [225]:
# bf = df_3[df_3['type'] == 'breakfast']
# lun = df_3[df_3['type'] == 'lunch']
# din = df_3[df_3['type'] == 'dinner']

In [300]:
meals = pd.DataFrame()
for j in df_3.columns:
    sub = []
    if j in ['Menu Items', 'type', 'restrictions']:
        pass
    else:
        for k in day.keys():
            for i in day[k]:
                a = df_3['Menu Items'].isin(list(i))
                choice = df_3[a].fillna(0)
                sub.append(sum(choice[j]))
        meals[j] = sub
types = []
for i in day.keys():
    types += [i] * len(day[i])
meals['type'] = types    

bf_opt = meals[meals['type'] == 'bf']
lun_opt = meals[meals['type'] == 'lun']
din_opt = meals[meals['type'] == 'din']

final_options = pd.DataFrame()
for i in range(len(bf_opt.index)):
    for j in range(len(lun_opt.index)):
        for k in range(len(din_opt.index)):
            option = pd.DataFrame()
            option = pd.concat([bf_opt.iloc[[i]],lun_opt.iloc[[j]],din_opt.iloc[[k]]],axis = 0)
            row = {}
            for l in option.columns:
                if l == 'type':
                    pass
                else:
                    row[l] = sum(option[l])
            row['combo'] = str(i) + str(j) + str(k)
            info = pd.DataFrame.from_dict(row, orient = 'index')
            info = info.transpose()
            final_options = pd.concat([final_options, info], axis = 0)
            
final_options.reset_index(drop=True, inplace=True) 
for i in final_options.columns:
    if i == 'combo':
        pass
    else:
        final_options[i] = pd.to_numeric(final_options[i])


,calories,water (mL),saturated_fat (g),trans_fat (g),cholesterol (mg),sodium (mg),total_carbs (g),dietary_fiber (g),total_sugars (g),protein (g),potassium (mg),calcium (mg),iron (mg),vitamin_d (mcg),combo
0,2961.0,0.0,70.8,1.6,474.2,3650.0,234.8,28.7,137.0,86.7,4587.6,868.0,13.8,3.7,000
1,2941.5,0.0,66.3,0.9,546.9,5598.5,192.4,18.7,106.7,119.9,3677.2,931.0,14.0,4.3,001
2,2739.0,0.0,45.9,0.2,347.8,4163.1,243.0,24.4,117.7,92.0,3420.8,835.6,15.0,2.4,002
3,2417.0,0.0,47.9,1.7,288.8,2291.2,233.3,31.4,126.7,48.6,5063.8,628.8,8.2,3.2,010
4,2397.5,0.0,43.4,1.0,361.5,4239.7,190.9,21.4,96.4,81.8,4153.4,691.8,8.4,3.8,011
5,2195.0,0.0,23.0,0.3,162.4,2804.3,241.5,27.1,107.4,53.9,3897.0,596.4,9.4,1.9,012
6,2659.0,0.0,54.4,1.7,368.8,2961.0,250.4,20.3,156.7,58.9,4029.6,529.0,6.8,3.5,020
7,2639.5,0.0,49.9,1.0,441.5,4909.5,208.0,10.3,126.4,92.1,3119.2,592.0,7.0,4.1,021
8,2437.0,0.0,29.5,0.3,242.4,3474.1,258.6,16.0,137.4,64.2,2862.8,496.6,8.0,2.2,022
9,3125.0,0.0,85.1,1.5,775.2,3937.0,179.2,25.5,79.4,91.5,3547.4,710.0,13.2,2.3,100


In [297]:
no_need = ['resident_id', 'restrictions']
~df_2.columns.isin(no_need)
comp = df_2[~df_2.columns.isin(no_need)]

KeyError: "None of [Index([(False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True)], dtype='object')] are in the [columns]"